# Phase 3, 4 & 5: Autoencoder + Ensemble + Evaluation\n
\n
## This notebook covers:\n
1. **Phase 3**: Autoencoder for anomaly detection\n
2. **Phase 4**: Ensemble fraud scoring system\n
3. **Phase 5**: Comprehensive evaluation and reporting

## Step 1: Import Libraries

In [ ]:
import numpy as np\n
import pandas as pd\n
import matplotlib.pyplot as plt\n
import seaborn as sns\n
from sklearn.metrics import (\n
    accuracy_score, precision_score, recall_score, f1_score,\n
    confusion_matrix, classification_report, roc_auc_score, roc_curve,\n
    precision_recall_curve, auc\n
)\n
import tensorflow as tf\n
from tensorflow import keras\n
from tensorflow.keras.models import Sequential, Model, load_model\n
from tensorflow.keras.layers import Dense, Input, Dropout\n
from tensorflow.keras.optimizers import Adam\n
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint\n
import joblib\n
import time\n
import warnings\n
warnings.filterwarnings('ignore')\n
\n
# Set random seeds\n
np.random.seed(42)\n
tf.random.set_seed(42)\n
\n
print('All libraries imported successfully!')

## Step 2: Load Data and Previous Models

In [ ]:
# Load preprocessed data\n
X_train = pd.read_csv('../../data/processed/X_train.csv')\n
X_test = pd.read_csv('../../data/processed/X_test.csv')\n
y_train = pd.read_csv('../../data/processed/y_train.csv').values.ravel()\n
y_test = pd.read_csv('../../data/processed/y_test.csv').values.ravel()\n
\n
print(f'Data loaded: Train {X_train.shape}, Test {X_test.shape}')

In [ ]:
# Load trained models\n
try:\n
    lstm_model = load_model('../../models/saved_models/lstm_model.keras')\n
    gru_model = load_model('../../models/saved_models/gru_model.keras')\n
    print('LSTM and GRU models loaded successfully!')\n
except:\n
    print('WARNING: LSTM/GRU models not found. Train them in notebook 02 first.')\n
    lstm_model = None\n
    gru_model = None

# ==== PHASE 3: AUTOENCODER MODEL ====\n
\n
## Step 3: Build Autoencoder\n
\n
Autoencoder trains on **normal transactions only**, then detects fraud as anomalies based on reconstruction error.

In [ ]:
# Extract only normal transactions for training\n
X_train_normal = X_train[y_train == 0]\n
X_test_normal = X_test[y_test == 0]\n
X_test_fraud = X_test[y_test == 1]\n
\n
print(f'Normal transactions (training): {X_train_normal.shape}')\n
print(f'Normal transactions (test): {X_test_normal.shape}')\n
print(f'Fraud transactions (test): {X_test_fraud.shape}')

In [ ]:
# Build Autoencoder architecture\n
input_dim = X_train.shape[1]\n
encoding_dim = 14\n
\n
# Encoder\n
input_layer = Input(shape=(input_dim,))\n
encoded = Dense(24, activation='relu')(input_layer)\n
encoded = Dropout(0.2)(encoded)\n
encoded = Dense(encoding_dim, activation='relu')(encoded)\n
\n
# Decoder\n
decoded = Dense(24, activation='relu')(encoded)\n
decoded = Dropout(0.2)(decoded)\n
decoded = Dense(input_dim, activation='linear')(decoded)\n
\n
# Autoencoder model\n
autoencoder = Model(inputs=input_layer, outputs=decoded)\n
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])\n
\n
autoencoder.summary()

## Step 4: Train Autoencoder

In [ ]:
# Train on normal transactions only\n
early_stop_ae = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)\n
checkpoint_ae = ModelCheckpoint('../../models/saved_models/autoencoder_model.keras', \n
                               save_best_only=True, monitor='val_loss')\n
\n
print('Training Autoencoder on normal transactions only...')\n
\n
history_ae = autoencoder.fit(\n
    X_train_normal, X_train_normal,\n
    validation_split=0.2,\n
    epochs=30,\n
    batch_size=256,\n
    callbacks=[early_stop_ae, checkpoint_ae],\n
    verbose=1\n
)\n
\n
print('\\nAutoencoder training completed!')

In [ ]:
# Plot training history\n
fig, axes = plt.subplots(1, 2, figsize=(14, 5))\n
\n
# Loss\n
axes[0].plot(history_ae.history['loss'], label='Training Loss')\n
axes[0].plot(history_ae.history['val_loss'], label='Validation Loss')\n
axes[0].set_title('Autoencoder - Loss', fontsize=14, fontweight='bold')\n
axes[0].set_xlabel('Epoch')\n
axes[0].set_ylabel('MSE Loss')\n
axes[0].legend()\n
axes[0].grid(alpha=0.3)\n
\n
# MAE\n
axes[1].plot(history_ae.history['mae'], label='Training MAE')\n
axes[1].plot(history_ae.history['val_mae'], label='Validation MAE')\n
axes[1].set_title('Autoencoder - MAE', fontsize=14, fontweight='bold')\n
axes[1].set_xlabel('Epoch')\n
axes[1].set_ylabel('MAE')\n
axes[1].legend()\n
axes[1].grid(alpha=0.3)\n
\n
plt.tight_layout()\n
plt.show()

## Step 5: Calculate Reconstruction Error

In [ ]:
# Predict (reconstruct)\n
X_test_reconstructed = autoencoder.predict(X_test)\n
\n
# Calculate reconstruction error (MSE per sample)\n
reconstruction_error = np.mean(np.square(X_test.values - X_test_reconstructed), axis=1)\n
\n
print(f'Reconstruction error calculated: {reconstruction_error.shape}')\n
print(f'Mean error: {reconstruction_error.mean():.6f}')\n
print(f'Std error: {reconstruction_error.std():.6f}')

In [ ]:
# Visualize reconstruction error distribution\n
fig, ax = plt.subplots(figsize=(12, 6))\n
\n
# Plot distributions\n
normal_errors = reconstruction_error[y_test == 0]\n
fraud_errors = reconstruction_error[y_test == 1]\n
\n
ax.hist(normal_errors, bins=50, alpha=0.7, label='Normal', color='#2ecc71', density=True)\n
ax.hist(fraud_errors, bins=50, alpha=0.7, label='Fraud', color='#e74c3c', density=True)\n
ax.set_xlabel('Reconstruction Error', fontsize=12)\n
ax.set_ylabel('Density', fontsize=12)\n
ax.set_title('Reconstruction Error Distribution', fontsize=14, fontweight='bold')\n
ax.legend()\n
ax.grid(alpha=0.3)\n
\n
plt.tight_layout()\n
plt.show()

## Step 6: Set Anomaly Threshold

In [ ]:
# Set threshold as 95th percentile of normal errors\n
threshold = np.percentile(normal_errors, 95)\n
print(f'Anomaly threshold (95th percentile): {threshold:.6f}')\n
\n
# Predict based on threshold\n
y_pred_ae = (reconstruction_error > threshold).astype(int)\n
\n
# Normalize reconstruction error to [0, 1] for ensemble\n
from sklearn.preprocessing import MinMaxScaler\n
scaler = MinMaxScaler()\n
y_pred_proba_ae = scaler.fit_transform(reconstruction_error.reshape(-1, 1)).ravel()

## Step 7: Evaluate Autoencoder

In [ ]:
# Metrics\n
print('=== AUTOENCODER RESULTS ===')\n
print(f'Accuracy: {accuracy_score(y_test, y_pred_ae):.4f}')\n
print(f'Precision: {precision_score(y_test, y_pred_ae):.4f}')\n
print(f'Recall: {recall_score(y_test, y_pred_ae):.4f}')\n
print(f'F1-Score: {f1_score(y_test, y_pred_ae):.4f}')\n
print(f'ROC-AUC: {roc_auc_score(y_test, y_pred_proba_ae):.4f}')\n
print('\\nClassification Report:')\n
print(classification_report(y_test, y_pred_ae, target_names=['Normal', 'Fraud']))\n
print('\\nConfusion Matrix:')\n
print(confusion_matrix(y_test, y_pred_ae))

# ==== PHASE 4: ENSEMBLE FRAUD SCORING ====\n
\n
## Step 8: Generate Ensemble Predictions

In [ ]:
# Get predictions from all models\n
if lstm_model and gru_model:\n
    # Reshape for LSTM/GRU\n
    X_test_seq = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))\n
    \n
    y_pred_proba_lstm = lstm_model.predict(X_test_seq).ravel()\n
    y_pred_proba_gru = gru_model.predict(X_test_seq).ravel()\n
    \n
    print('LSTM and GRU predictions generated!')\n
    \n
    # Weighted ensemble\n
    alpha = 0.35  # LSTM weight\n
    beta = 0.35   # GRU weight\n
    gamma = 0.30  # Autoencoder weight\n
    \n
    ensemble_score = (alpha * y_pred_proba_lstm + \n
                     beta * y_pred_proba_gru + \n
                     gamma * y_pred_proba_ae)\n
    \n
    print(f'\\nEnsemble weights: LSTM={alpha}, GRU={beta}, Autoencoder={gamma}')\n
    print(f'Ensemble scores generated: {ensemble_score.shape}')\n
else:\n
    print('WARNING: Using Autoencoder only for predictions')\n
    ensemble_score = y_pred_proba_ae

## Step 9: Optimize Ensemble Threshold

In [ ]:
# Find optimal threshold using F1-score\n
thresholds = np.linspace(0, 1, 100)\n
f1_scores = []\n
\n
for thresh in thresholds:\n
    y_pred_temp = (ensemble_score > thresh).astype(int)\n
    f1_scores.append(f1_score(y_test, y_pred_temp))\n
\n
optimal_threshold = thresholds[np.argmax(f1_scores)]\n
print(f'Optimal ensemble threshold: {optimal_threshold:.4f}')\n
print(f'Max F1-Score: {max(f1_scores):.4f}')

In [ ]:
# Plot threshold optimization\n
fig, ax = plt.subplots(figsize=(10, 6))\n
ax.plot(thresholds, f1_scores, linewidth=2, color='#3498db')\n
ax.axvline(optimal_threshold, color='#e74c3c', linestyle='--', linewidth=2, \n
          label=f'Optimal Threshold = {optimal_threshold:.3f}')\n
ax.set_xlabel('Threshold', fontsize=12)\n
ax.set_ylabel('F1-Score', fontsize=12)\n
ax.set_title('Ensemble Threshold Optimization', fontsize=14, fontweight='bold')\n
ax.legend()\n
ax.grid(alpha=0.3)\n
plt.tight_layout()\n
plt.show()

## Step 10: Evaluate Ensemble Model

In [ ]:
# Final predictions\n
y_pred_ensemble = (ensemble_score > optimal_threshold).astype(int)\n
\n
# Metrics\n
print('=== ENSEMBLE MODEL RESULTS ===')\n
print(f'Accuracy: {accuracy_score(y_test, y_pred_ensemble):.4f}')\n
print(f'Precision: {precision_score(y_test, y_pred_ensemble):.4f}')\n
print(f'Recall: {recall_score(y_test, y_pred_ensemble):.4f}')\n
print(f'F1-Score: {f1_score(y_test, y_pred_ensemble):.4f}')\n
print(f'ROC-AUC: {roc_auc_score(y_test, ensemble_score):.4f}')\n
print('\\nClassification Report:')\n
print(classification_report(y_test, y_pred_ensemble, target_names=['Normal', 'Fraud']))\n
print('\\nConfusion Matrix:')\n
print(confusion_matrix(y_test, y_pred_ensemble))

# ==== PHASE 5: COMPREHENSIVE EVALUATION ====\n
\n
## Step 11: Compare All Models

In [ ]:
# Load baseline models for comparison\n
try:\n
    lr_model = joblib.load('../../models/saved_models/logistic_regression.pkl')\n
    rf_model = joblib.load('../../models/saved_models/random_forest.pkl')\n
    \n
    y_pred_proba_lr = lr_model.predict_proba(X_test)[:, 1]\n
    y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]\n
    \n
    y_pred_lr = lr_model.predict(X_test)\n
    y_pred_rf = rf_model.predict(X_test)\n
    \n
    has_baseline = True\n
    print('Baseline models loaded successfully!')\n
except:\n
    print('WARNING: Baseline models not found')\n
    has_baseline = False

In [ ]:
# Create comprehensive comparison\n
if lstm_model and gru_model and has_baseline:\n
    models_list = ['Logistic Reg', 'Random Forest', 'LSTM', 'GRU', 'Autoencoder', 'Ensemble']\n
    \n
    accuracy_list = [\n
        accuracy_score(y_test, y_pred_lr),\n
        accuracy_score(y_test, y_pred_rf),\n
        accuracy_score(y_test, (y_pred_proba_lstm > 0.5).astype(int)),\n
        accuracy_score(y_test, (y_pred_proba_gru > 0.5).astype(int)),\n
        accuracy_score(y_test, y_pred_ae),\n
        accuracy_score(y_test, y_pred_ensemble)\n
    ]\n
    \n
    precision_list = [\n
        precision_score(y_test, y_pred_lr),\n
        precision_score(y_test, y_pred_rf),\n
        precision_score(y_test, (y_pred_proba_lstm > 0.5).astype(int)),\n
        precision_score(y_test, (y_pred_proba_gru > 0.5).astype(int)),\n
        precision_score(y_test, y_pred_ae),\n
        precision_score(y_test, y_pred_ensemble)\n
    ]\n
    \n
    recall_list = [\n
        recall_score(y_test, y_pred_lr),\n
        recall_score(y_test, y_pred_rf),\n
        recall_score(y_test, (y_pred_proba_lstm > 0.5).astype(int)),\n
        recall_score(y_test, (y_pred_proba_gru > 0.5).astype(int)),\n
        recall_score(y_test, y_pred_ae),\n
        recall_score(y_test, y_pred_ensemble)\n
    ]\n
    \n
    f1_list = [\n
        f1_score(y_test, y_pred_lr),\n
        f1_score(y_test, y_pred_rf),\n
       f1_score(y_test, (y_pred_proba_lstm > 0.5).astype(int)),\n
        f1_score(y_test, (y_pred_proba_gru > 0.5).astype(int)),\n
        f1_score(y_test, y_pred_ae),\n
        f1_score(y_test, y_pred_ensemble)\n
    ]\n
    \n
    # Create comparison DataFrame\n
    results_df = pd.DataFrame({\n
        'Model': models_list,\n
        'Accuracy': accuracy_list,\n
        'Precision': precision_list,\n
        'Recall': recall_list,\n
        'F1-Score': f1_list\n
    })\n
    \n
    print('\\n=== COMPREHENSIVE MODEL COMPARISON ===')\n
    print(results_df.to_string(index=False))\n
    \n
    # Save results\n
    results_df.to_csv('../../reports/model_comparison.csv', index=False)\n
    print('\\nResults saved to reports/model_comparison.csv')

## Step 12: Visualization Dashboard

In [ ]:
# Comprehensive visualization\n
if lstm_model and gru_model and has_baseline:\n
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))\n
    \n
    # Accuracy\n
    axes[0, 0].barh(models_list, accuracy_list, color='#3498db')\n
    axes[0, 0].set_xlabel('Accuracy', fontsize=12)\n
    axes[0, 0].set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')\n
    axes[0, 0].set_xlim([0.95, 1.0])\n
    axes[0, 0].grid(alpha=0.3, axis='x')\n
    \n
    # Precision\n
    axes[0, 1].barh(models_list, precision_list, color='#e67e22')\n
    axes[0, 1].set_xlabel('Precision', fontsize=12)\n
    axes[0, 1].set_title('Model Precision Comparison', fontsize=14, fontweight='bold')\n
    axes[0, 1].set_xlim([0, 1])\n
    axes[0, 1].grid(alpha=0.3, axis='x')\n
    \n
    # Recall\n
    axes[1, 0].barh(models_list, recall_list, color='#2ecc71')\n
    axes[1, 0].set_xlabel('Recall', fontsize=12)\n
    axes[1, 0].set_title('Model Recall Comparison', fontsize=14, fontweight='bold')\n
    axes[1, 0].set_xlim([0, 1])\n
    axes[1, 0].grid(alpha=0.3, axis='x')\n
    \n
    # F1-Score\n
    axes[1, 1].barh(models_list, f1_list, color='#9b59b6')\n
    axes[1, 1].set_xlabel('F1-Score', fontsize=12)\n
    axes[1, 1].set_title('Model F1-Score Comparison', fontsize=14, fontweight='bold')\n
    axes[1, 1].set_xlim([0, 1])\n
    axes[1, 1].grid(alpha=0.3, axis='x')\n
    \n
    plt.tight_layout()\n
    plt.savefig('../../reports/model_comparison.png', dpi=300, bbox_inches='tight')\n
    plt.show()\n
    \n
    print('Comparison chart saved to reports/model_comparison.png')

## Step 13: ROC & PR Curves

In [ ]:
# ROC Curves\n
if lstm_model and gru_model and has_baseline:\n
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))\n
    \n
    # ROC Curve\n
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_proba_lr)\n
    fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)\n
    fpr_lstm, tpr_lstm, _ = roc_curve(y_test, y_pred_proba_lstm)\n
    fpr_gru, tpr_gru, _ = roc_curve(y_test, y_pred_proba_gru)\n
    fpr_ae, tpr_ae, _ = roc_curve(y_test, y_pred_proba_ae)\n
    fpr_ens, tpr_ens, _ = roc_curve(y_test, ensemble_score)\n
    \n
    axes[0].plot(fpr_lr, tpr_lr, label=f'Logistic Reg (AUC={roc_auc_score(y_test, y_pred_proba_lr):.3f})', linewidth=2)\n
    axes[0].plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC={roc_auc_score(y_test, y_pred_proba_rf):.3f})', linewidth=2)\n
    axes[0].plot(fpr_lstm, tpr_lstm, label=f'LSTM (AUC={roc_auc_score(y_test, y_pred_proba_lstm):.3f})', linewidth=2)\n
    axes[0].plot(fpr_gru, tpr_gru, label=f'GRU (AUC={roc_auc_score(y_test, y_pred_proba_gru):.3f})', linewidth=2)\n
    axes[0].plot(fpr_ae, tpr_ae, label=f'Autoencoder (AUC={roc_auc_score(y_test, y_pred_proba_ae):.3f})', linewidth=2)\n
    axes[0].plot(fpr_ens, tpr_ens, label=f'Ensemble (AUC={roc_auc_score(y_test, ensemble_score):.3f})', linewidth=3, color='red')\n
    axes[0].plot([0, 1], [0, 1], 'k--', label='Random')\n
    axes[0].set_xlabel('False Positive Rate', fontsize=12)\n
    axes[0].set_ylabel('True Positive Rate', fontsize=12)\n
    axes[0].set_title('ROC Curve - All Models', fontsize=14, fontweight='bold')\n
    axes[0].legend(fontsize=9)\n
    axes[0].grid(alpha=0.3)\n
    \n
    # Precision-Recall Curve\n
    prec_lr, rec_lr, _ = precision_recall_curve(y_test, y_pred_proba_lr)\n
    prec_rf, rec_rf, _ = precision_recall_curve(y_test, y_pred_proba_rf)\n
    prec_lstm, rec_lstm, _ = precision_recall_curve(y_test, y_pred_proba_lstm)\n
    prec_gru, rec_gru, _ = precision_recall_curve(y_test, y_pred_proba_gru)\n
    prec_ae, rec_ae, _ = precision_recall_curve(y_test, y_pred_proba_ae)\n
    prec_ens, rec_ens, _ = precision_recall_curve(y_test, ensemble_score)\n
    \n
    axes[1].plot(rec_lr, prec_lr, label=f'Logistic Reg (AP={auc(rec_lr, prec_lr):.3f})', linewidth=2)\n
    axes[1].plot(rec_rf, prec_rf, label=f'Random Forest (AP={auc(rec_rf, prec_rf):.3f})', linewidth=2)\n
    axes[1].plot(rec_lstm, prec_lstm, label=f'LSTM (AP={auc(rec_lstm, prec_lstm):.3f})', linewidth=2)\n
    axes[1].plot(rec_gru, prec_gru, label=f'GRU (AP={auc(rec_gru, prec_gru):.3f})', linewidth=2)\n
    axes[1].plot(rec_ae, prec_ae, label=f'Autoencoder (AP={auc(rec_ae, prec_ae):.3f})', linewidth=2)\n
    axes[1].plot(rec_ens, prec_ens, label=f'Ensemble (AP={auc(rec_ens, prec_ens):.3f})', linewidth=3, color='red')\n
    axes[1].set_xlabel('Recall', fontsize=12)\n
    axes[1].set_ylabel('Precision', fontsize=12)\n
    axes[1].set_title('Precision-Recall Curve - All Models', fontsize=14, fontweight='bold')\n
    axes[1].legend(fontsize=9)\n
    axes[1].grid(alpha=0.3)\n
    \n
    plt.tight_layout()\n
    plt.savefig('../../reports/roc_pr_curves.png', dpi=300, bbox_inches='tight')\n
    plt.show()\n
    \n
    print('ROC & PR curves saved to reports/roc_pr_curves.png')

## Step 14: Latency Measurement (Simulated Real-Time)

In [ ]:
# Measure inference latency\n
print('=== INFERENCE LATENCY MEASUREMENT ===')\n
\n
# Sample 1000 transactions\n
sample_size = 1000\n
X_sample = X_test.iloc[:sample_size]\n
X_sample_seq = X_sample.values.reshape((sample_size, 1, X_sample.shape[1]))\n
\n
latencies = {}\n
\n
if has_baseline:\n
    # Logistic Regression\n
    start = time.time()\n
    _ = lr_model.predict(X_sample)\n
    latencies['Logistic Reg'] = (time.time() - start) * 1000 / sample_size\n
    \n
    # Random Forest\n
    start = time.time()\n
    _ = rf_model.predict(X_sample)\n
    latencies['Random Forest'] = (time.time() - start) * 1000 / sample_size\n
\n
if lstm_model:\n
    # LSTM\n
    start = time.time()\n
    _ = lstm_model.predict(X_sample_seq, verbose=0)\n
    latencies['LSTM'] = (time.time() - start) * 1000 / sample_size\n
\n
if gru_model:\n
    # GRU\n
    start = time.time()\n
    _ = gru_model.predict(X_sample_seq, verbose=0)\n
    latencies['GRU'] = (time.time() - start) * 1000 / sample_size\n
\n
# Autoencoder\n
start = time.time()\n
_ = autoencoder.predict(X_sample, verbose=0)\n
latencies['Autoencoder'] = (time.time() - start) * 1000 / sample_size\n
\n
# Ensemble\n
start = time.time()\n
if lstm_model and gru_model:\n
    _ = lstm_model.predict(X_sample_seq, verbose=0)\n
    _ = gru_model.predict(X_sample_seq, verbose=0)\n
_ = autoencoder.predict(X_sample, verbose=0)\n
latencies['Ensemble'] = (time.time() - start) * 1000 / sample_size\n
\n
print('\\nAverage Latency per Transaction (ms):')\n
for model, latency in latencies.items():\n
    print(f'{model}: {latency:.3f} ms')

In [ ]:
# Plot latency\n
fig, ax = plt.subplots(figsize=(10, 6))\n
models = list(latencies.keys())\n
lats = list(latencies.values())\n
\n
bars = ax.barh(models, lats, color='#e74c3c')\n
ax.set_xlabel('Latency (ms)', fontsize=12)\n
ax.set_title('Model Inference Latency (per transaction)', fontsize=14, fontweight='bold')\n
ax.grid(alpha=0.3, axis='x')\n
\n
# Add value labels\n
for bar in bars:\n
    width = bar.get_width()\n
    ax.text(width, bar.get_y() + bar.get_height()/2, \n
           f'{width:.2f} ms', ha='left', va='center', fontsize=10)\n
\n
plt.tight_layout()\n
plt.savefig('../../reports/latency_comparison.png', dpi=300, bbox_inches='tight')\n
plt.show()\n
\n
print('Latency chart saved to reports/latency_comparison.png')

## Final Summary\n
\n
### ✅ All Phases Completed:\n
\n
**Phase 1**: Data preprocessing + baseline models\n
**Phase 2**: LSTM + GRU deep learning models\n
**Phase 3**: Autoencoder anomaly detection\n
**Phase 4**: Ensemble fraud scoring system\n
**Phase 5**: Comprehensive evaluation + reporting\n
\n
### 🔥 Key Achievements:\n
- Built 6 fraud detection models\n
- Comprehensive comparison across all metrics\n
- Optimized ensemble scoring system\n
- Measured real-time inference latency\n
- Generated research-grade visualizations\n
\n
### 📊 Next Steps:\n
- Implement Kafka + Spark real-time streaming (Phase 4 extension)\n
- Deploy models in production environment\n
- Add explainability (SHAP/LIME)\n
- Create PowerPoint presentation\n
\n
### ⚠️ Limitations:\n
- Simulated environment (not production-grade)\n
- Single-node execution (not distributed)\n
- PCA features (limited interpretability)\n
- Static dataset (not truly streaming)